In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Deep NN as classifier
class DNN(nn.Module):
    def __init__(self, use_pretrained = True, nb_outs = 1):
        super(DNN, self).__init__()
        
        # Mobilenet model to finetune for our problem
        self.net = models.mobilenet_v2(pretrained = use_pretrained, progress=True)
        
        # Altering last layer to our need
        self.net.classifier = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(1280, 1, bias=False))
        
    def forward(self, x):
        x = self.net(x)
        return x
  
# Model init
model = DNN()

# Move model to device
model = model.to(device)

# Freeze inititial layers of mobilenet
i = 0
for name, param in model.named_parameters():
    print(i, name)
    if i < 135:
        param.requires_grad = False
    i+=1

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth



0 net.features.0.0.weight
1 net.features.0.1.weight
2 net.features.0.1.bias
3 net.features.1.conv.0.0.weight
4 net.features.1.conv.0.1.weight
5 net.features.1.conv.0.1.bias
6 net.features.1.conv.1.weight
7 net.features.1.conv.2.weight
8 net.features.1.conv.2.bias
9 net.features.2.conv.0.0.weight
10 net.features.2.conv.0.1.weight
11 net.features.2.conv.0.1.bias
12 net.features.2.conv.1.0.weight
13 net.features.2.conv.1.1.weight
14 net.features.2.conv.1.1.bias
15 net.features.2.conv.2.weight
16 net.features.2.conv.3.weight
17 net.features.2.conv.3.bias
18 net.features.3.conv.0.0.weight
19 net.features.3.conv.0.1.weight
20 net.features.3.conv.0.1.bias
21 net.features.3.conv.1.0.weight
22 net.features.3.conv.1.1.weight
23 net.features.3.conv.1.1.bias
24 net.features.3.conv.2.weight
25 net.features.3.conv.3.weight
26 net.features.3.conv.3.bias
27 net.features.4.conv.0.0.weight
28 net.features.4.conv.0.1.weight
29 net.features.4.conv.0.1.bias
30 net.features.4.conv.1.0.weight
31 net.feature

In [3]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

# Visulize batch images
def visulize_batch(x):
    for i in range(x.shape[0]):
        img = x[i, :, :, :].numpy().transpose((1,2,0))
        img = (img*std_nums) + mean_nums
        img = np.clip(img, 0, 1)
        plt.imshow(img)
        plt.show()    

# Data transform pipeline for training set
train_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees = 30, translate=(.2, .2), scale=(0.8, 1.2), shear=[-10, 10, -10, 10]),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_nums, std_nums)
])

# Data transform pipeline for validation set
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_nums, std_nums)
])

# Dataset init for train and val
train_dataset = datasets.ImageFolder("../input/face-mask-12k-images-dataset/Face Mask Dataset/Train", transform=train_transform)
val_dataset = datasets.ImageFolder("../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation", transform=val_transform)

# Dataloader to batch input data
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle=True, num_workers=4)

In [4]:
# for x, target in train_loader:
#     print(target)
#     visulize_batch(x)
#     break

In [5]:
# Binary cross entropy loss
criterion = nn.BCEWithLogitsLoss()

# Adam optimizer to update model params (Only some of last layers)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

# LR scheduler to decrease lr if val_loss stops improving
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)

In [6]:
# Number of epochs to train for
num_epochs = 10

# Keep track of min val loss
min_val_loss = np.inf

# Training loop
for i in range(num_epochs):
    train_loss = 0.
    val_loss = 0.
    
    # Switch model to training mode
    model.train()
    
    # Looping through train set
    for inputs, labels in train_loader:
        # Move batch data to device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Clear out previous gradients
        optimizer.zero_grad()
        
        # Forward pass
        preds = model(inputs).squeeze(1)
        labels = labels.type_as(preds)
        
        # Compute loss given labels and predictions
        loss = criterion(preds, labels)
        train_loss += (loss.item()*inputs.size(0))
        
        # Backpropagate to calculate gradients w.r.t. loss
        loss.backward()
        
        # Make weight updates
        optimizer.step()

    # Switch model to evaluation mode
    model.eval()
    
    # Never compute gradients while evaluating
    with torch.no_grad():
        
        # Looping through val set to find val_loss
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            preds = model(inputs).squeeze(1)
            labels = labels.type_as(preds)

            loss = criterion(preds, labels)

            val_loss += (loss.item()*inputs.size(0))
      
    # Calculate avg. train and val loss
    train_loss /= len(train_dataset)
    val_loss /= len(val_dataset)
    
    # If val_loss is minimum than seen before save the model
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'Mobilenet_v2_Mask_Detection.pt')
    
    # Change lr if val_loss not improving given init params
    scheduler.step(val_loss)
        
    print('Epoch {}:\tTrain Loss: {:.6f}\tVal Loss: {:.6f}'.format(i+1, train_loss, val_loss))

Epoch 1:	Train Loss: 0.035990	Val Loss: 0.007323
Epoch 2:	Train Loss: 0.011301	Val Loss: 0.001499
Epoch 3:	Train Loss: 0.008590	Val Loss: 0.000372
Epoch 4:	Train Loss: 0.007054	Val Loss: 0.003165
Epoch 5:	Train Loss: 0.011228	Val Loss: 0.000410
Epoch 6:	Train Loss: 0.004974	Val Loss: 0.000011
Epoch 7:	Train Loss: 0.008755	Val Loss: 0.000918
Epoch 8:	Train Loss: 0.007316	Val Loss: 0.000025
Epoch 9:	Train Loss: 0.004920	Val Loss: 0.000080
Epoch 10:	Train Loss: 0.003205	Val Loss: 0.000017
